In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from omegaconf import OmegaConf

In [2]:
with open("../cross_eval_configs/cross_eval_seeds.txt") as f:
    seeds = list(map(int, f.readlines()))

print(seeds)

[12345, 67890, 11121, 31415, 92653, 58979, 32384, 62643, 38327, 95028]


In [3]:
runs_path = Path("../runs/ppo")
run_basename = "final_trunk_thesis_"
all_runs = list(runs_path.glob(f"{run_basename}*"))
all_runs.sort()
print(all_runs)

[PosixPath('../runs/ppo/final_trunk_thesis_training_13_2025-05-09_19-54-16'), PosixPath('../runs/ppo/final_trunk_thesis_training_16_2025-05-09_19-49-38'), PosixPath('../runs/ppo/final_trunk_thesis_training_16_2025-05-09_21-01-13'), PosixPath('../runs/ppo/final_trunk_thesis_training_1_2025-05-09_19-54-19'), PosixPath('../runs/ppo/final_trunk_thesis_training_26_2025-05-09_19-54-17'), PosixPath('../runs/ppo/final_trunk_thesis_training_32_2025-05-09_19-50-22'), PosixPath('../runs/ppo/final_trunk_thesis_training_32_2025-05-09_21-01-16'), PosixPath('../runs/ppo/final_trunk_thesis_training_420_2025-05-09_19-54-16'), PosixPath('../runs/ppo/final_trunk_thesis_training_42_2025-05-09_19-48-57'), PosixPath('../runs/ppo/final_trunk_thesis_training_42_2025-05-09_21-01-13'), PosixPath('../runs/ppo/final_trunk_thesis_training_64_2025-05-09_19-51-13'), PosixPath('../runs/ppo/final_trunk_thesis_training_64_2025-05-09_21-01-16'), PosixPath('../runs/ppo/final_trunk_thesis_training_666_2025-05-09_19-54-33'

In [4]:
from flipper_training.experiments.ppo.eval import get_eval_rollout, log_from_eval_rollout, PPOExperimentConfig
import contextlib
from tqdm import tqdm
import pickle
import os

In [5]:
n_robots_eval = 16

In [ ]:
results_per_run = {}
null_stream = open(os.devnull, "w+")
for run in tqdm(all_runs, desc="Runs", position=0):
    cfg = OmegaConf.load(run / "config.yaml")
    cfg["num_robots"] = n_robots_eval
    # cfg["engine_compile_opts"] = None
    cfg["objective_opts"]["cache_size"] = 10
    policy_wt = run / "weights" / "policy_final.pth"
    vecnorm_wt = run / "weights" / "vecnorm_final.pth"
    if not policy_wt.exists() or not vecnorm_wt.exists():
        print(f"Policy or vecnorm weights not found for {run}")
        continue
    result_logs = []
    for seed in tqdm(seeds, desc="Seeds", position=1):
        cfg["seed"] = seed
        with contextlib.redirect_stdout(null_stream), contextlib.redirect_stderr(null_stream):
            _, rollout = get_eval_rollout(PPOExperimentConfig(**cfg), policy_wt, vecnorm_wt)
            rollout_log = log_from_eval_rollout(rollout)
        result_logs.append(rollout_log)
    results_per_run[run] = result_logs
    print(f"Run {run} done")

with open(f"../cross_eval_results/{run_basename}results_training_distr.pkl", "wb") as f:
    pickle.dump(results_per_run, f)

Runs:   0%|          | 0/14 [00:00<?, ?it/s]

                 Environment Summary                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Key ┃                       Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    Number of robots │                          16 │
│        Observations │ LocalStateVector, Heightmap │
│              Reward │               PotentialGoal │
│           Objective │               TrunkCrossing │
│   Physics frequency │                   142.86 Hz │
│   Engine iters/step │                           4 │
│ Effective frequency │                    35.71 Hz │
└─────────────────────┴─────────────────────────────┘

2025-05-10 19:19:55,947 [torchrl][INFO] check_env_specs succeeded!


     Policy Parameter Counts     
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃        Component ┃ Parameters ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│    Actor Encoder │     32,032 │
│       Actor Head │     13,456 │
│    Value Encoder │     32,032 │
│       Value Head │     12,737 │
│              --- │        --- │
│      Total Actor │     45,488 │
│      Total Value │     44,769 │
│              --- │        --- │
│ Total Parameters │     90,257 │
└──────────────────┴────────────┘

                 Environment Summary                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Key ┃                       Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    Number of robots │                          16 │
│        Observations │ LocalStateVector, Heightmap │
│              Reward │               PotentialGoal │
│           Objective │               TrunkCrossing │
│   Physics frequency │                   142.86 Hz │
│   Engine iters/step │                           4 │
│ Effective frequency │                    35.71 Hz │
└─────────────────────┴─────────────────────────────┘

...

Runs:   7%|▋         | 1/14 [03:36<46:51, 216.24s/it]

Run ../runs/ppo/final_trunk_thesis_training_13_2025-05-09_19-54-16 done


                 Environment Summary                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Key ┃                       Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    Number of robots │                          16 │
│        Observations │ LocalStateVector, Heightmap │
│              Reward │               PotentialGoal │
│           Objective │               TrunkCrossing │
│   Physics frequency │                   142.86 Hz │
│   Engine iters/step │                           4 │
│ Effective frequency │                    35.71 Hz │
└─────────────────────┴─────────────────────────────┘

2025-05-10 19:23:22,519 [torchrl][INFO] check_env_specs succeeded!
Runs:   7%|▋         | 1/14 [03:46<49:08, 226.80s/it]


FileNotFoundError: [Errno 2] No such file or directory: '../runs/ppo/final_trunk_thesis_training_16_2025-05-09_19-49-38/weights/policy_final.pth'